In [94]:
%pip install ninjabook tabulate --force-reinstall

  Using cached ninjabook-0.1.5-cp38-abi3-macosx_11_0_arm64.whl.metadata (429 bytes)
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
Using cached ninjabook-0.1.5-cp38-abi3-macosx_11_0_arm64.whl (215 kB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
  Attempting uninstall: tabulate
    Found existing installation: tabulate 0.9.0
    Uninstalling tabulate-0.9.0:
      Successfully uninstalled tabulate-0.9.0
  Attempting uninstall: ninjabook
    Found existing installation: ninjabook 0.1.5
    Uninstalling ninjabook-0.1.5:
      Successfully uninstalled ninjabook-0.1.5

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [95]:
import csv
from ninjabook import Orderbook, Event


events = []
with open("../data/norm_book_data_300k.csv", newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        event = Event(
            timestamp= int(row['timestamp']),
            seq=int(row['seq']),
            is_trade=bool(row['is_trade']=='1'),
            is_buy=bool(row['is_buy']=='1'),
            price=float(row['price']),
            size=float(row['size'])
        )
        events.append(event)

len(events)

300000

In [101]:
from tabulate import tabulate 

data = []

ob = Orderbook(0.01)

for event in events[:200_000]:
    bbo = ob.process_stream_bbo(event)
    if bbo != None:
        best_bid_px = bbo[0].price if bbo[0] is not None else None
        best_bid_sz = bbo[0].size if bbo[0] is not None else None
        best_ask_px = bbo[1].price if bbo[1] is not None else None
        best_ask_sz = bbo[1].size if bbo[1] is not None else None
        mid = ob.midprice()
        data.append([ob.last_updated, ob.last_sequence, best_bid_sz, best_bid_px, mid, best_ask_px, best_ask_sz])


headers = ["ts", "seq", "best_bid_sz", "best_bid_px", "mid", "best_ask_px", "best_ask_sz"]

print(tabulate(data[-5:], headers=headers, tablefmt="grid"))


+------------------+--------+---------------+---------------+---------+---------------+---------------+
|               ts |    seq |   best_bid_sz |   best_bid_px |     mid |   best_ask_px |   best_ask_sz |
+==================+========+===============+===============+=========+===============+===============+
| 1575161162310275 | 199955 |      0.019027 |       7490.55 | 7491.39 |       7492.22 |      0.204504 |
+------------------+--------+---------------+---------------+---------+---------------+---------------+
| 1575161162409739 | 199968 |      0.019027 |       7490.55 | 7491.35 |       7492.16 |      0.019353 |
+------------------+--------+---------------+---------------+---------+---------------+---------------+
| 1575161162509896 | 199980 |      0.019027 |       7490.55 | 7491.33 |       7492.1  |      0.3      |
+------------------+--------+---------------+---------------+---------+---------------+---------------+
| 1575161162610363 | 199987 |      0.019027 |       7490.55 | 74

In [97]:
from tabulate import tabulate 

data = []

ob = Orderbook(0.01)

for event in events[:200_000]:
    ob.process(event)
    
top_asks = ob.top_asks(5)
top_bids = ob.top_bids(5)

# for visualization purposes only
sorted_asks = sorted(top_asks, key=lambda x: x.price, reverse=True)

for ask in sorted_asks:
    data.append([None, None, None, ask.price, ask.size])
    
data.append([None, None, ob.midprice(), None, None])

for bid in top_bids:
    data.append([bid.size, bid.price, None, None, None])
    

headers = ["best_bid_sz", "best_bid_px", "mid", "best_ask_px", "best_ask_sz"]

print("last sequence:", ob.last_sequence)
print(tabulate(data, headers=headers, tablefmt="grid"))


last sequence: 199999
+---------------+---------------+---------+---------------+---------------+
|   best_bid_sz |   best_bid_px |     mid |   best_ask_px |   best_ask_sz |
+===============+===============+=========+===============+===============+
|               |               |         |       7492.05 |      0.019353 |
+---------------+---------------+---------+---------------+---------------+
|               |               |         |       7492.03 |      0.211905 |
+---------------+---------------+---------+---------------+---------------+
|               |               |         |       7492.02 |      4        |
+---------------+---------------+---------+---------------+---------------+
|               |               |         |       7492.01 |      0.149845 |
+---------------+---------------+---------+---------------+---------------+
|               |               |         |       7491.99 |      0.214587 |
+---------------+---------------+---------+---------------+-------